<a href="https://colab.research.google.com/github/a07458666/TBrainNLP/blob/master/3_BERT_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kashgari==1.1.5
!pip install tensorflow-gpu==1.15.0
!pip install keras-applications==1.0.8
!pip install keras==2.3.1

In [ ]:
import os
import re
import numpy as np
import pandas as pd
from google.colab import drive
from tqdm import tqdm

drive.mount('/content/drive')
os.chdir(r"/content/drive/My Drive/tbrainData/")

## Load Data

In [ ]:
all_content = np.load('all_content.npy', allow_pickle = True)
all_BIO = np.load('all_BIO.npy', allow_pickle = True)
all_article_index = np.load('all_article_index.npy', allow_pickle = True)

In [ ]:
dataset = pd.DataFrame({'content': all_content, 'BIO': all_BIO, 'all_article_index': all_article_index})
dataset["NoName"] = dataset["BIO"].apply(lambda x: int(all(np.array(x) == "O")))
dataset.head()

## split train / val / test dataset

In [ ]:
def train_validate_test_split(input_df, train_percent=.6, validate_percent=.2, seed=None):
    input_list = np.unique(input_df['all_article_index']).tolist()
    np.random.seed(seed)
    perm = np.random.permutation(range(len(input_list))).tolist()
    m = len(input_list)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = list(input_list[i] for i in perm[:train_end])
    validate = list(input_list[i] for i in perm[train_end:validate_end])
    test = list(input_list[i] for i in perm[validate_end:])
        
    # -- train data --
    train_data = [dataset[dataset['all_article_index'].isin(train)]['content'].tolist(), dataset[dataset['all_article_index'].isin(train)]['NoName'].tolist(), dataset[dataset['all_article_index'].isin(train)]['all_article_index'].tolist()]

    # -- valid data --
    valid_data = [dataset[dataset['all_article_index'].isin(validate)]['content'].tolist(), dataset[dataset['all_article_index'].isin(validate)]['NoName'].tolist(), dataset[dataset['all_article_index'].isin(validate)]['all_article_index'].tolist()]

    # -- test data --
    test_data = [dataset[dataset['all_article_index'].isin(test)]['content'].tolist(),  dataset[dataset['all_article_index'].isin(test)]['NoName'].tolist(), dataset[dataset['all_article_index'].isin(test)]['all_article_index'].tolist()]

    return train_data, valid_data, test_data

In [ ]:
train_data, valid_data, test_data = train_validate_test_split(input_df=dataset, train_percent=.5, validate_percent=.3, seed=777)
train_x, train_y, train_idx =  train_data[0], train_data[1], train_data[2]
valid_x, valid_y, valid_idx =  valid_data[0], valid_data[1], valid_data[2]
test_x, test_y, test_idx =  test_data[0], test_data[1], test_data[2]
print(f"train data count: {len(train_x)}")
print(f"validate data count: {len(valid_x)}")
print(f"test data count: {len(test_x)}")

## Download BERT model

In [ ]:
!wget -nc "https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip"
!unzip -o "chinese_L-12_H-768_A-12.zip"

## Model

In [ ]:
import kashgari
from kashgari.tasks.classification import BiLSTM_Model

import logging
logging.basicConfig(level='DEBUG')

model = BiLSTM_Model()
model.fit(train_x, train_y, valid_x, valid_y, epochs=1)

model.save('saved_classification_model')

loaded_model = kashgari.utils.load_model('saved_classification_model')

loaded_model.predict(test_x[:10])
loaded_model.evaluate(test_x, test_y)